<a href="https://colab.research.google.com/github/Chuck2Win/attention/blob/attention_follow_up_1/attention_follow_up_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from gensim.models import Word2Vec
import nltk
import os
from google.colab import files
from google.colab import drive
from matplotlib import pyplot as plt
import json
# google drive와 연동
drive.mount('/content/gdrive')
print(os.getcwd())
# os.chdir('..') ..은 상위 폴더로 이동
os.chdir('./gdrive/My Drive/attention_from_master_')
print(os.getcwd())


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/attention_from_master_


FileNotFoundError: ignored

In [50]:
! pip install sentencepiece
import sentencepiece as spm
# testing
vocab_file = "./kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
inputs=[]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(line)
  print(pieces)
  print(ids)
  print() # 단어를 subword 단위로 잘 쪼개는것을 확인할 수 있습니다.

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3099, 3582, 604, 3596, 684, 4009, 3593, 109, 4189, 206, 3951, 3754, 3584]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[3228, 86, 3961, 3917, 3598, 264, 669, 1940, 3658, 777, 1442, 3788, 3754, 4243]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3099, 3585, 208, 3599, 53, 3826, 3590, 3608, 3594, 235, 3865, 3597, 61, 3817, 448, 3676, 3754, 3584]



In [51]:
print(len(vocab)) # 8007
print(len(ids))

8007
18


In [52]:
# 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0)을 추가 해 줌
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
print(inputs)
print(inputs.shape) # 3, 18 -> 가장 긴 문장을 기준으로 padding 해준다.

tensor([[3099, 3582,  604, 3596,  684, 4009, 3593,  109, 4189,  206, 3951, 3754,
         3584,    0,    0,    0,    0,    0],
        [3228,   86, 3961, 3917, 3598,  264,  669, 1940, 3658,  777, 1442, 3788,
         3754, 4243,    0,    0,    0,    0],
        [3099, 3585,  208, 3599,   53, 3826, 3590, 3608, 3594,  235, 3865, 3597,
           61, 3817,  448, 3676, 3754, 3584]])
torch.Size([3, 18])


In [0]:
# positional encoding을 해줘야한다.
def positional_encoding(seq_len,embedding_dim):
    # input shape : [n,seq_len]
    #output=torch.zeros((seq_len+1,embedding_dim),device=device)
    output=torch.zeros((seq_len,embedding_dim),device=device)
    def for_insert(output):
        m,n=output.shape
        for i in range(m):
            for j in range(n):
                output[i,j]=i/10000**(2*j/embedding_dim)
        return output
    pe=for_insert(output)
    pe[:,0::2]=torch.sin(pe[:,0::2])
    pe[:,1::2]=torch.cos(pe[:,1::2])
    # pe[0,:]=0. 없애고 진행해보자
    return pe

In [54]:
device='cpu'
positional_encoding(64,300)

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.8943e-01,  7.7356e-01,  ...,  1.0000e+00,
          1.1307e-08,  1.0000e+00],
        [ 9.0930e-01, -3.0515e-01,  9.8045e-01,  ...,  1.0000e+00,
          2.2613e-08,  1.0000e+00],
        ...,
        [-9.6612e-01,  6.8334e-01, -5.1718e-01,  ...,  1.0000e+00,
          6.8970e-07,  1.0000e+00],
        [-7.3918e-01, -1.8701e-01, -9.8982e-01,  ...,  1.0000e+00,
          7.0101e-07,  1.0000e+00],
        [ 1.6736e-01, -9.0380e-01, -7.3737e-01,  ...,  1.0000e+00,
          7.1232e-07,  1.0000e+00]])

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.8943e-01,  7.7356e-01,  ...,  1.0000e+00,
          1.1307e-08,  1.0000e+00],
        [ 9.0930e-01, -3.0515e-01,  9.8045e-01,  ...,  1.0000e+00,
          2.2613e-08,  1.0000e+00],
        ...,
        [-9.6612e-01,  6.8334e-01, -5.1718e-01,  ...,  1.0000e+00,
          6.8970e-07,  1.0000e+00],
        [-7.3918e-01, -1.8701e-01, -9.8982e-01,  ...,  1.0000e+00,
          7.0101e-07,  1.0000e+00],
        [ 1.6736e-01, -9.0380e-01, -7.3737e-01,  ...,  1.0000e+00,
          7.1232e-07,  1.0000e+00]])

In [55]:
# 이를 토대로 positional embedding을 구한다.
'''
위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. 
학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.
입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.
input값 중 pad(0)값을 찾습니다.
positions값중 pad부분은 0으로 변경 합니다. (mask를 활용)
positions값에 해당하는 embedding값을 구합니다
'''

'\n위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. \n학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.\n입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.\ninput값 중 pad(0)값을 찾습니다.\npositions값중 pad부분은 0으로 변경 합니다. (mask를 활용)\npositions값에 해당하는 embedding값을 구합니다\n'

'\n위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. \n학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.\n입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.\ninput값 중 pad(0)값을 찾습니다.\npositions값중 pad부분은 0으로 변경 합니다. (mask를 활용)\npositions값에 해당하는 embedding값을 구합니다\n'

In [56]:
############################# input embedding ##########################################
n_vocab=len(vocab)
embedding=128
seq_len=64 # 아싸리 크게 하면 되겠네. 딱 타이트하게 하지 말고...
Embedding=nn.Embedding(n_vocab,embedding,padding_idx=0)
Embedding.weight.required_grad=False
inputs_embedding=Embedding(inputs) 
############################# positional embedding ##########################################
pos_encoding=positional_encoding(seq_len,embedding)
positional_embedding=nn.Embedding.from_pretrained(pos_encoding,freeze=True)
positions=torch.arange(inputs.size(1)).expand(inputs.size(0),inputs.size(1))+1 # inputs size : n,seq_len
print(positions)
pos_mask=inputs.eq(0) # mask shape : n,seq_len : 3, 18
positions.masked_fill_(pos_mask,0)  # (어차피 0에는 무엇을 곱해도 0이 되니깐)->아니지. 그리고 1을 더했다.. 그전에
print(positions)
pos_embedding=positional_embedding(positions)
print(pos_embedding[1]) # 보게 되면 0에 해당하는 부분은 0,1,0,1로 반복되고 있다.

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])
tensor([[ 8.4147e-01,  6.4791e-01,  6.8156e-01,  ...,  1.0000e+00,
          1.3335e-08,  1.0000e+00],
        [ 9.0930e-01, -1.6044e-01,  9.9748e-01,  ...,  1.0000e+00,
          2.6670e-08,  1.0000e+00],
        [ 1.4112e-01, -8.5580e-01,  7.7827e-01,  ...,  1.0000e+00,
          4.0006e-08,  1.0000e+00],
        ...,
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.00

In [57]:
# positional embedding과 token embedding을 더해야지
input_sums=inputs_embedding+pos_embedding 
# anyway, padding이 된 부분에 해당되는 부분이 0 값이 된 것은 아니다.. -> 그렇기 때문에...
inputs[0]
input_sums[0]

tensor([[ 1.7790,  1.4683, -0.1738,  ...,  1.8388, -1.5923,  1.0434],
        [ 2.8732,  0.5689,  0.5324,  ...,  1.9610, -1.2025,  1.6823],
        [-0.4929, -1.9724,  0.8430,  ..., -0.4396,  0.8397,  1.8953],
        ...,
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000]],
       grad_fn=<SelectBackward>)

tensor([[-0.9317, -0.9484,  1.4497,  ...,  0.3678,  0.4071,  0.3801],
        [ 3.0645,  0.3304,  1.3181,  ...,  2.2471, -1.7370,  1.3173],
        [ 1.3605, -1.7109, -0.1601,  ..., -0.0892,  0.1210,  0.7888],
        ...,
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000]],
       grad_fn=<SelectBackward>)

In [58]:
Q=input_sums # shape : 3, 18, 128
K=input_sums # inputs shape : 3, 18
V=input_sums
# inputs.shape : batch_size, seq_len
attention_mask=inputs.eq(0).unsqueeze(1).expand(Q.size(0),Q.size(1),K.size(1)) # shape : 3, 18, 18
# print(get_attention_pad_mask(Q,K))
print(attention_mask[0].shape)
print(attention_mask[0])

torch.Size([18, 18])
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, 

In [0]:
class scaled_dot_product_attention(nn.Module): # 고민 요소
    # 나의 초안에서 attention 함수를 차용해 옴
    def __init__(self):
        super().__init__()
               
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        scores=Q.matmul(K.transpose(-2,-1))/torch.sqrt(torch.tensor(Q.shape[-1]).float())
        # 여기에서 attention mask는 padding이 된 부분을 0으로 처리한 것
        # attention mask shape : n,seq_len,seq_len
        # 00011
        # 00011
        # 00011
        # 00011
        # 00011 
        # 요론 모양
        scores.masked_fill_(attention_mask,-1e-9)
        weight=F.softmax(scores,dim=-1)
        # weight : n, seq_len, seq_len
        attention_score=weight.matmul(V)
        # attention score : n,seq_len,dv
        return attention_score, weight

In [0]:
# 나는 multi head attention을 안했었는데, 병렬로 처리하기 위해선 더 좋으니깐
# 쩐다...★
class multi_head_attention(nn.Module): # 고민 요소
    # 나의 초안에서 attention 함수를 차용해 옴
    def __init__(self,batch_size,n_head,d_head,d_k):
        super().__init__()
        self.batch_size=batch_size
        self.n_head=n_head
        self.d_head=d_head
        self.d_k=d_k
        self.W_Q=nn.Linear(d_k,n_head*d_head)
        self.W_K=nn.Linear(d_k,n_head*d_head)
        self.W_V=nn.Linear(d_k,n_head*d_head)
        self.W_0=nn.Linear(n_head*d_head,n_head*d_head) # n_head*d_head = embedding
        self.sdpa=scaled_dot_product_attention()
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        # attention_mask shape : [n,seq_len,seq_len]
        qs=self.W_Q(Q).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2) # ★
        ks=self.W_K(K).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        vs=self.W_V(V).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        attention_mask=attention_mask.unsqueeze(1).repeat(1,self.n_head,1,1) # attention_mask shape : [n,self.n_head,seq_len,seq_len] <- mask도 동일하게 multi head로 확장시켜 나가야지
        context,weight=self.sdpa(qs,ks,vs,attention_mask) # context shape : n, n_head, seq_len, d_head
        context=context.transpose(1,2).reshape(self.batch_size,-1,self.n_head*self.d_head)
        output=self.W_0(context)
        return output, weight

In [61]:
mm=multi_head_attention(3,2,64,128)
mm.forward(Q,K,V,attention_mask)[0].shape

torch.Size([3, 18, 128])

torch.Size([3, 18, 128])

In [0]:
# masked multi_head attention
def get_attention_decoder_mask(seq):
    # seq : batch_size, query_seq_len
    subsequent_mask=torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0),seq.size(1),seq.size(1))
    # mask : batch_size, seq_len, seq_len
    mask=subsequent_mask.triu(diagonal=1)
    return mask   

In [66]:
attn_dec_mask=get_attention_decoder_mask(inputs)
attn_dec_mask.shape
attn_dec_mask[0]

tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [70]:
attn_pad_mask=inputs.eq(0).unsqueeze(1).expand(Q.size(0),Q.size(1),K.size(1))
print(attn_pad_mask[1])

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True, 

In [0]:
# pad mask와 decoder mask를 합쳐야 한다.
attention_decoder_mask=get_attention_decoder_mask(inputs)
attn_mask=torch.gt((attn_pad_mask+attention_decoder_mask),0) # element wise로 0보다 크면 True

In [78]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_hidn):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_hidn * 4, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_hidn * 4, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        # (bs, n_seq, d_hidn)
        return output

tensor([[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 

In [0]:
# ? torch.gt()
attn_mask=torch.gt((attention_pad_mask+attention_decoder_mask),0) # torch.gt(input,other)=> input>other인 경우 element wise
print(attn_mask[0].long())

In [0]:
class Config(dict):
    __getitem__=dict.__getitem__
    __setitem__=dict.__setitem__

In [0]:
config=Config({"n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidden": 256,
    "padding_idx": 0,
    "d_feedforward": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})

In [0]:
config